## Import libraries

In [1]:
import requests
import re
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

## Unpickle .pkl file back to dataframe

In [2]:
#Raw data
#data = pd.read_pickle("sources/df_articles.pkl")
#data = pd.read_pickle("sources/df_length_of_articles.pkl")

#Data after preprocessing
data = pd.read_pickle("sources/df_articles_ap.pkl")

#data.reset_index(inplace = True, drop = True)
data

,body,id,published_at,raw_body,title,url,source.name,source_name_cred
0,Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- Pavo...,769528,2020-01-07T12:26:43.000000+0100,<p>Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- P...,Koledníci zo Šale získali na rozvojové projekt...,https://www.hlavnespravy.sk/kolednici-ziskali-...,hlavnespravy.sk,1
1,Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/AP-P...,878049,2020-04-23T06:01:14.000000+0200,<p>Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/A...,Dvaja migranti sa ľahko zranili pri streľbe v ...,https://www.hlavnespravy.sk/dvaja-migranti-sa-...,hlavnespravy.sk,1
2,Esslingen 5. februára 2018 (TASR/HSP/Foto:TASR...,726743,2018-02-05T16:46:32.000000+0100,<p>Esslingen 5. februára 2018 (TASR/HSP/Foto:T...,V Nemecku štvorčlenná rodina prišla o život zr...,https://www.hlavnespravy.sk/v-nemecku-stvorcle...,hlavnespravy.sk,1
3,\nInformační doplněk k hlavnímu proudu\nSource...,773010,2020-01-14T08:28:39.000000+0100,"<p><a href=""https://prima.iprima.cz/ranni-kavi...",14. 1. 2020 &#8211; Svět při úterní ranní kávi...,https://slobodnyvysielac.sk/2020/01/14/14-1-20...,slobodnyvysielac.sk,0
4,Bratislava 24. júla 2020 (HSP/Združenia sloven...,850448,2020-07-24T09:00:50.000000+0200,"<p><a href=""https://www.hlavnespravy.sk/rozhov...","Rozhovor s doc. Jánom Dudášom, DrSc. o jeho tr...",https://slobodnyvysielac.sk/2020/07/24/rozhovo...,slobodnyvysielac.sk,0
...,...,...,...,...,...,...,...,...
76773,Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/AP/K...,739558,2018-10-03T19:32:17.000000+0200,<p>Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/A...,Novým vietnamským prezidentom bude generálny t...,https://www.hlavnespravy.sk/novym-vietnamskym-...,hlavnespravy.sk,1
76774,Bratislava 10. marca 2020 (TASR/HSP/Foto:SITA-...,870796,2020-03-10T11:21:09.000000+0100,<p>Bratislava 10. marca 2020 (TASR/HSP/Foto:SI...,Miroslav Lajčák sa zúčastní na ministerskom st...,https://www.hlavnespravy.sk/miroslav-lajcak-sa...,hlavnespravy.sk,1
76775,Vždy a tak aj pri vzdelaní máte na výber. Buď ...,471352,2019-12-07T09:40:33.000000+0100,"<div>\n\n <p class=""perex""><str...",Škola pre mimoriadne nadané deti (a kam chodia...,https://romanhutira.blog.sme.sk/c/522598/skola...,sme.sk,1
76776,Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP-Ra...,749541,2019-04-06T16:52:48.000000+0200,<p>Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP...,Zástupcovia žltých viest z celého Francúzska r...,https://www.hlavnespravy.sk/zastupcovia-zltych...,hlavnespravy.sk,1


## Load konspiratori.sk, mediabiasfactcheck.com and merge these lists


In [3]:
with open('sources/zoznam_konspiratori.txt') as f:
    lines = f.readlines()
   #print(lines)
    
list_konspiratori = [x.strip() for x in lines] 

with open('sources/zoznam_mediabiasfactcheck.txt') as f:
    lines = f.readlines()
   #print(lines)
    
list_mediabiasfactcheck = [x.strip() for x in lines] 

list_konspiratori.extend(list_mediabiasfactcheck)
list_konspiratori

['slovenskaiskra.sk',
 'hlavnespravy.sk',
 'vigilantcitizen.com',
 'protiprudu.org',
 'aeronet.news',
 'biblik.sk',
 'orgo-net.blogspot.sk',
 'aeronet.cz',
 'conspi.cz',
 'nwoo.org',
 'geneticka-vakcina.com',
 'arfa.cz',
 'exopolitika.cz',
 'badatel.net',
 'czechfreepress.cz',
 'qanon.sk',
 'zemavek.sk',
 'stratenaeuropa.sk',
 'bnw-nwo.wz.cz',
 'protiprud.sk',
 'protiproud.cz',
 'blokprotiislamizaci.cz',
 'afinabul.blog.cz',
 'prvopodstata.com',
 'stalo-se.cz',
 'slobodavockovani.sk',
 'magnificat.sk',
 'nekorektni-tv.cz',
 'veksvetla.cz',
 'sho.sk',
 'tadesco.org',
 'nadhlad.com',
 'pravdive.eu',
 'bezpolitickekorektnosti.cz',
 'freeglobe.cz',
 'stop5g.cz',
 'svobodnenoviny.eu',
 'vedomec.com',
 'anti-illuminati.wbl.sk',
 'leva-net.webnode.cz',
 'vlasteneckenoviny.cz',
 'novarepublika.cz',
 'rukojmi.cz',
 'rozhladna.org',
 'yournewswire.com',
 'kralicianora.org',
 'slobodnyvysielac.sk',
 'ezopress.sk',
 'prekladyodlesa.sk',
 'pokec24.cz',
 'obcianskytribunal.sk',
 'infowars.cz',
 'ide

## For test

In [ ]:
for i in range(len(data)):
    for website in list_konspiratori:
         domain = re.findall("https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+",data['raw_body'][i])
         print(domain)
        

In [ ]:
txt = data['raw_body'][3]
regex = "HSP"
#txt = re.compile(r'HSP', re.I)

# domains = re.findall(p,txt)
# domains = list(set(domains))

print(txt)
# for domain in domains:
#     print(domain)

text = 'HSP'

if text in txt:
    print("Found!")
else:
    print("Not found!")


## Check usage of non credible sources in article + add new collumn

In [ ]:
new_col_source = []

for article_num in range(len(data)):
    domains = []
    regex = "https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+"
    p = re.compile(regex)
    
    raw_body = data['raw_body'][article_num]
    print(raw_body)
    
    domains = re.findall(p,raw_body)
    domains = list(set(domains))
    print(domains)
    
    #check list if contains URL which consist of source.name, if yes remove them 
    domains = [source for source in domains if not data['source.name'][article_num] in source]
        
    
    print(domains)
    print(data['source.name'][article_num])
    score = 1
    for website in list_konspiratori:
        if score == 0:
            break
        for domain in domains:
            if website in domain:
                score = 0
                break
            else:
                score = 1
    new_col_source.append(score)
    score = 0

data['reference_cred'] = new_col_source
print(data[['source.name', 'source_name_cred', 'reference_cred']])

In [ ]:
#data.to_pickle("sources/data_credible_sources.pkl")

In [4]:
data_automated = pd.read_pickle("sources/data_credible_sources.pkl")

In [ ]:
print("Počet článkov s nekredibilnými referenciami {}".format(data_automated['reference_cred'].value_counts()[0]))
print("Počet článkov s kredibilnými referenciami {}".format(data_automated['reference_cred'].value_counts()[1]))

print("Počet článkov s nekredibilnou doménou {}".format(data_automated['source_name_cred'].value_counts()[0]))
print("Počet článkov s kredibilnou doménou {}".format(data_automated['source_name_cred'].value_counts()[1]))

In [ ]:
pd.set_option('display.max_rows', None)
data[(data['source.name'] == 'hlavnespravy.sk') & (data['source_name_cred'] == 1)]

## Load ids of credible_reference subset 100 ids of manually anotated articles

In [5]:
#load subset of randomly choosed subset of 100 articles
with open("sources/manually_annotated_subset/subset_100_ids.txt", "r") as file:
    readline=file.read().split("\n")
    print(readline)

#load labels for articles above
with open("sources/manually_annotated_subset/source_reference_subset_100_labels.txt", "r") as file:
    labels=file.read().split("\n")
    print(labels)
    
#load features
df_features = pd.read_pickle("sources/data_credible_sources.pkl")
df_features = df_features['reference_cred']
df_features

['8', '120', '17', '3529', '18', '128', '19', '7775', '45619', '2184', '134', '15385', '5464', '129', '21', '25', '29', '36', '163', '45', '134', '52', '180', '65', '3187', '74', '48798', '1420', '77', '78', '47204', '79', '84', '140', '21917', '151', '9549', '163', '19347', '33333', '66', '100', '164', '12909', '122', '133', '1284', '135', '136', '144', '167', '5337', '146', '14052', '64612', '67489', '251', '189', '152', '39310', '46', '9376', '162', '45083', '190', '197', '215', '218', '3', '225', '236', '4', '239', '11', '242', '245', '28', '247', '60706', '254', '273', '37', '299', '12217', '301', '305', '39', '22035', '37348', '51', '335', '60744', '60', '336', '337', '86', '30208', '357', '376', '112']
['1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', 

0        1
1        1
2        1
3        1
4        0
        ..
76773    1
76774    1
76775    1
76776    1
76777    1
Name: reference_cred, Length: 76778, dtype: int64

In [6]:
my_list = []
my_array = []
itter = 0

for i in readline:
    my_array.append(i)
    my_array.append(data_automated['id'][int(i)])
    my_array.append(data_automated['url'][int(i)])
    my_array.append(labels[itter])
    my_list.append(my_array)
    itter += 1
    
    my_array = []

df_100 = pd.DataFrame(my_list, columns =['poradie', 'id', 'url', 'labels']) 

In [7]:
data_automated.reset_index(inplace = True, drop = True)
df_100 = pd.merge(df_100, data_automated)
df_100

,poradie,id,url,labels,body,published_at,raw_body,title,source.name,source_name_cred,reference_cred
0,8,917791,https://finweb.hnonline.sk/ekonomika/2225220-r...,1,Predstavitelia gastrosektora sú pripravení usk...,2020-10-07T13:29:00.000000+0200,<div><p>Predstavitelia gastrosektora s&#250; p...,Reštaurácie volajú o pomoc a chcú nižšiu DPH. ...,hnonline.sk,1,1
1,120,447040,https://slobodnyvysielac.sk/2019/02/05/co-ked-...,0,\n Lokálne dôsledky zmien vo svete môžu mať do...,2019-02-05T08:24:53.000000+0100,"\n<figure class=""wp-block-audio""><audio contro...",Čo keď vyhrá Harabin. „Zvolia“ nám v Bruseli n...,slobodnyvysielac.sk,0,0
2,17,784498,https://hnonline.sk/svet/2085577-este-sa-neda-...,1,Svetová zdravotnícka organizácia (WHO) vo štvr...,2020-02-06T19:31:00.000000+0100,<div><p>Svetov&#225; zdravotn&#237;cka organiz...,"Ešte sa nedá povedať, že epidémia koronavírusu...",hnonline.sk,1,1
3,3529,1372855,https://www.slovenskaiskra.sk/zhorsuju-sa-viac...,1,\nZhoršujú sa viac-menej všetky ukazovatele. M...,2021-03-02T21:59:33.000000+0100,"<div id=""fb-root""></div>\n<p>Zhoršujú sa viac-...","Zhoršujú sa viac-menej všetky ukazovatele, uvi...",slovenskaiskra.sk,0,1
4,18,1079914,https://blog.sme.sk/koprda/spolocnost/ockovaci...,1,Písmo: A - | A +\n\nDo hazardných hier dávajú ...,2021-08-15T16:42:39.000000+0200,"<div><p class=""article-font-size"">\n <p cla...",Očkovacia lotéria: Čo môže priniesť a kde sú j...,sme.sk,1,1
...,...,...,...,...,...,...,...,...,...,...,...
95,86,390753,https://www.bajecnezdravie.sk/vyskum-potvrdzuj...,0,Medicínsky výskum opätovne potvrdzuje protirak...,2019-04-09T21:51:37.000000+0200,<p>Medicínsky výskum opätovne potvrdzuje proti...,"Výskum znova potvrdzuje, že injekčný vitamín C...",bajecnezdravie.sk,0,1
96,30208,1296707,https://www.badatel.net/novozelandski-vedci-po...,0,"\n(Mary Villareal, Natural News) Novozélandskí...",2022-02-09T19:14:15.000000+0100,"<p><img loading=""lazy"" class=""aligncenter wp-i...","Novozélandskí vedci potvrdili, že vakcíny Pfiz...",badatel.net,0,0
97,357,816045,https://zuzanajankajova.blog.sme.sk/c/533631/k...,1,Dnes by som Vám rada ukázala cestu ako robiť v...,2020-04-16T02:45:33.000000+0200,"<div>\n\n <p class=""perex"">Dnes...","Keď Nájdeme V Sebe Pokoj, Všetko Pôjde Ľahšie ...",sme.sk,1,1
98,376,806406,https://peterkolesar2.blog.sme.sk/c/531295/vyz...,1,Obchodné reťazce sa musia hneď začať pripravov...,2020-03-24T18:52:12.000000+0100,"<div>\n\n <p class=""perex"">Obch...",Výzva obchodným reťazcom: Zaveďte výdaj pred o...,sme.sk,1,1


## Decision tree with cross validation

In [8]:
col_names = ['reference_cred']

X = df_100[col_names].values.reshape(-1, 1) # Features
y = df_100.labels


depth = []
for i in range(2,10):
    clf = DecisionTreeClassifier(max_depth=i)
    #5-fold cross validation 
    scores = cross_val_score(estimator=clf, X=X, y=y, cv=5, n_jobs=4)
    depth.append((i,scores.mean()))
print(depth)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

clf = DecisionTreeClassifier( max_depth=2)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[(2, 0.86), (3, 0.86), (4, 0.86), (5, 0.86), (6, 0.86), (7, 0.86), (8, 0.86), (9, 0.86)]
Accuracy: 0.8
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.84      0.94      0.89        17

    accuracy                           0.80        20
   macro avg       0.42      0.47      0.44        20
weighted avg       0.72      0.80      0.76        20



In [9]:
y_pred = clf.predict(df_features.values.reshape(-1, 1))
y_pred = pd.DataFrame(y_pred, columns =['label'])
y_pred

data_automated = pd.concat([data_automated, y_pred], axis=1)
data_automated

,body,id,published_at,raw_body,title,url,source.name,source_name_cred,reference_cred,label
0,Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- Pavo...,769528,2020-01-07T12:26:43.000000+0100,<p>Šaľa 7. januára 2020 (TASR/HSP/Foto:TASR- P...,Koledníci zo Šale získali na rozvojové projekt...,https://www.hlavnespravy.sk/kolednici-ziskali-...,hlavnespravy.sk,1,1,1
1,Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/AP-P...,878049,2020-04-23T06:01:14.000000+0200,<p>Atény 23. apríla 2020 (TASR/HSP/Foto:TASR/A...,Dvaja migranti sa ľahko zranili pri streľbe v ...,https://www.hlavnespravy.sk/dvaja-migranti-sa-...,hlavnespravy.sk,1,1,1
2,Esslingen 5. februára 2018 (TASR/HSP/Foto:TASR...,726743,2018-02-05T16:46:32.000000+0100,<p>Esslingen 5. februára 2018 (TASR/HSP/Foto:T...,V Nemecku štvorčlenná rodina prišla o život zr...,https://www.hlavnespravy.sk/v-nemecku-stvorcle...,hlavnespravy.sk,1,1,1
3,\nInformační doplněk k hlavnímu proudu\nSource...,773010,2020-01-14T08:28:39.000000+0100,"<p><a href=""https://prima.iprima.cz/ranni-kavi...",14. 1. 2020 &#8211; Svět při úterní ranní kávi...,https://slobodnyvysielac.sk/2020/01/14/14-1-20...,slobodnyvysielac.sk,0,1,1
4,Bratislava 24. júla 2020 (HSP/Združenia sloven...,850448,2020-07-24T09:00:50.000000+0200,"<p><a href=""https://www.hlavnespravy.sk/rozhov...","Rozhovor s doc. Jánom Dudášom, DrSc. o jeho tr...",https://slobodnyvysielac.sk/2020/07/24/rozhovo...,slobodnyvysielac.sk,0,0,0
...,...,...,...,...,...,...,...,...,...,...
76773,Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/AP/K...,739558,2018-10-03T19:32:17.000000+0200,<p>Hanoj 3. októbra 2018 (TASR/HSP/Foto:TASR/A...,Novým vietnamským prezidentom bude generálny t...,https://www.hlavnespravy.sk/novym-vietnamskym-...,hlavnespravy.sk,1,1,1
76774,Bratislava 10. marca 2020 (TASR/HSP/Foto:SITA-...,870796,2020-03-10T11:21:09.000000+0100,<p>Bratislava 10. marca 2020 (TASR/HSP/Foto:SI...,Miroslav Lajčák sa zúčastní na ministerskom st...,https://www.hlavnespravy.sk/miroslav-lajcak-sa...,hlavnespravy.sk,1,1,1
76775,Vždy a tak aj pri vzdelaní máte na výber. Buď ...,471352,2019-12-07T09:40:33.000000+0100,"<div>\n\n <p class=""perex""><str...",Škola pre mimoriadne nadané deti (a kam chodia...,https://romanhutira.blog.sme.sk/c/522598/skola...,sme.sk,1,1,1
76776,Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP-Ra...,749541,2019-04-06T16:52:48.000000+0200,<p>Paríž 6. apríla 2019 (TASR/HSP/Foto:TASR/AP...,Zástupcovia žltých viest z celého Francúzska r...,https://www.hlavnespravy.sk/zastupcovia-zltych...,hlavnespravy.sk,1,1,1


In [ ]:
data_automated.to_pickle("sources/indicators_final_dataset/reference.pkl") 